# Лабораторная работа №1
## Задание:
1. Найти набор данных (датасет) для классификации удовлетворяющий следующим условиям: более 10 000 строк, более 20 столбцов, разные типы в столбцах, обязательно наличие целевого признака (таргета).
2. Провести классификацию найденного датасета, методом к- ближайших соседей. В формате Markdown писать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

### Подключение библиотек


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

### Загрузка данных в датасет


In [2]:
Dataset = pd.read_csv('property_assessment.csv')
dataset_rows = Dataset.shape[0]
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10321 entries, 0 to 10320
Data columns (total 46 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   PID                  10321 non-null  int64  
 1   CM_ID                10321 non-null  float64
 2   GIS_ID               10321 non-null  int64  
 3   ST_NUM               10321 non-null  float64
 4   ST_NAME              10321 non-null  object 
 5   CITY                 10321 non-null  object 
 6   ZIP_CODE             10321 non-null  float64
 7   BLDG_SEQ             10321 non-null  int64  
 8   NUM_BLDGS            10321 non-null  int64  
 9   LUC                  10321 non-null  int64  
 10  LU                   10321 non-null  object 
 11  LU_DESC              10321 non-null  object 
 12  BLDG_TYPE            10321 non-null  object 
 13  OWN_OCC              10321 non-null  object 
 14  OWNER                10321 non-null  object 
 15  MAIL_STREET_ADDRESS  10321 non-null 

### Создаём искомый столбец

In [3]:
label_encoder = LabelEncoder()
Dataset['COND_ENC'] = label_encoder.fit_transform(Dataset['OVERALL_COND'])

### Нормализация данных

In [4]:
Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']] = Dataset[['TOTAL_VALUE', 'LAND_VALUE', 'BLDG_VALUE']].applymap(lambda x: x.replace(',', '') if isinstance(x, str) else x).apply(pd.to_numeric, errors='coerce')
numeric_columns = Dataset.select_dtypes(include='number').columns.tolist()
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)
Dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10321 entries, 0 to 10320
Data columns (total 25 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PID          10321 non-null  int64  
 1   CM_ID        10321 non-null  float64
 2   GIS_ID       10321 non-null  int64  
 3   ST_NUM       10321 non-null  float64
 4   ZIP_CODE     10321 non-null  float64
 5   BLDG_SEQ     10321 non-null  int64  
 6   NUM_BLDGS    10321 non-null  int64  
 7   LUC          10321 non-null  int64  
 8   RES_FLOOR    10321 non-null  float64
 9   GROSS_AREA   10321 non-null  float64
 10  LIVING_AREA  10321 non-null  float64
 11  LAND_VALUE   10321 non-null  int64  
 12  BLDG_VALUE   10321 non-null  int64  
 13  SFYI_VALUE   10321 non-null  int64  
 14  TOTAL_VALUE  10321 non-null  int64  
 15  YR_BUILT     10321 non-null  float64
 16  YR_REMODEL   10321 non-null  float64
 17  BED_RMS      10321 non-null  float64
 18  FULL_BTH     10321 non-null  float64
 19  HLF_

### Поиск лучшей метрики и количества соседей

In [5]:
X = Dataset.drop(['COND_ENC'], axis=1)
y = Dataset['COND_ENC']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

knn = KNeighborsClassifier()

param_grid = {'n_neighbors': range(3, 20),
              'metric': ['euclidean', 'manhattan', 'chebyshev', 'minkowski']}

grid_search = GridSearchCV(knn, param_grid, cv=2, scoring='accuracy', n_jobs = 4)

grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
best_metric = best_params['metric']
best_k = best_params['n_neighbors']

accuracies = grid_search.cv_results_['mean_test_score']
accuracies = np.array(accuracies).reshape(len(param_grid['n_neighbors']), len(param_grid['metric']))

print(f"Лучшие параметры: Количество соседей = {best_k}, Лучшая метрика = {best_metric}")


Лучшие параметры: Количество соседей = 4, Лучшая метрика = manhattan


#### Повторное обучение на лучших метриках и количестве соседей

In [6]:
knn = KNeighborsClassifier(n_neighbors=best_k, metric=best_metric)

knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели с {best_k} соседями и метрикой {best_metric} : {accuracy}')
report = classification_report(y_test, y_pred, zero_division = 1)
print(report)

Точность модели с 4 соседями и метрикой manhattan : 0.7873577137321386
              precision    recall  f1-score   support

           0       0.82      0.94      0.88      3050
           1       0.54      0.47      0.50        62
           2       0.44      0.32      0.37        60
           3       1.00      0.00      0.00         1
           4       0.65      0.34      0.45       915
           5       0.50      0.20      0.28        41

    accuracy                           0.79      4129
   macro avg       0.66      0.38      0.41      4129
weighted avg       0.77      0.79      0.76      4129

